In [ ]:
#https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/

#import sys
#!conda install --yes --prefix {sys.prefix} r2pipe

import sys
!/Applications/Cutter.app/Contents/Frameworks/Python.framework/Versions/3.6/bin/python3
#install --prefix=/Applications/Cutter.app/Contents/Frameworks/Python.framework/Versions/3.6/ r2pipe


#/A/C/C/F/P/V/3/l/python3.7 pip3 install --prefix=(pwd)/Frameworks/Python.framework/Versions/3.6/ r2pipe


In [ ]:
#!/usr/bin/env python

import r2pipe
import json
import struct
import cutter
import re
import base64
from pprint import pprint, pformat

IP_MATCHER = re.compile("(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}(?:[:]\d+)?)")
URL_MATCHER = re.compile('(?:(?:https?|ftp|file)://|www\.|ftp\.)[-A-Z0-9+&@#/%=~_|$?!:,.]*[A-Z0-9+&@#/%=~_|$]', re.IGNORECASE)
EMAIL_MATCHER = re.compile('([A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4})', re.IGNORECASE)

def regex_matcher(matcher):
    return lambda st: matcher.findall(st)

def path_matcher(st):
    if st.startswith('/'):
        return [st]
    else:
        return []
    
def contains_matcher(s):
    return lambda st: [st] if s in st else []
                
matchers = [regex_matcher(IP_MATCHER), regex_matcher(URL_MATCHER), regex_matcher(EMAIL_MATCHER), path_matcher, contains_matcher('\\e['), contains_matcher('HTTP')]

def print_s(s, r):
    print('0x{:08x} 0x{:08x} {:10} {:4} {:10} {}'.format(s.get('paddr'), s.get('vaddr'), s.get('type'), s.get('length'), s.get('section'), r))

cutter.cmd('aaa')

strings = json.loads(cutter.cmd('izj'))
for s in strings:
    try:
        st = base64.b64decode(s.get('string')).decode(s.get('type'))

        for matcher in matchers:
            matches = matcher(st)
            for match in matches: 
                print_s (s, match)
    except ValueError as e:
        print(e)
        continue
    except LookupError as e:
        print(e)
        continue

cutter.refresh()
